---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [81]:
import pandas as pd
import numpy as np
from scipy import stats
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [120]:
def get_list_of_university_towns():
    LOUT = pd.read_table('university_towns.txt', header=None)
    DFOUT = pd.DataFrame(columns=["State", "RegionName"])
    Cur_State="US"
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )'''
    for i in range (0,len(LOUT)):
        if "[edit]" in LOUT.iloc[i][0]:
            Cur_State=LOUT.iloc[i][0]
        else:
            New_Row = pd.DataFrame([[Cur_State, LOUT.iloc[i][0]]], columns=["State", "RegionName"])
            DFOUT=DFOUT.append(New_Row, ignore_index=True)
    '''The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    DFOUT['RegionName'] = DFOUT['RegionName'].str.partition(" (")[0]
    DFOUT['RegionName'] = DFOUT['RegionName'].str.partition("\n")[0]
    DFOUT['State'] = DFOUT['State'].str.partition("[")[0]
    #DFOUT['State'] = DFOUT['State'].str.replace("\[.*\]","")
    #DFOUT['RegionName'] = DFOUT['RegionName'].str.replace("\n","")
    #DFOUT['RegionName'] = DFOUT['RegionName'].str.replace("\(.*\)","")
    
    return DFOUT

get_list_of_university_towns()

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo
5,Alabama,Troy
6,Alabama,Tuscaloosa
7,Alabama,Tuskegee
8,Alaska,Fairbanks
9,Arizona,Flagstaff


In [4]:
def get_recession_start():
    #df=pd.read_excel("gdplev.xls")
    df=pd.read_excel("gdplev.xls", skiprows=219, header=None)
    del (df[0],df[1],df[2],df[3],df[5],df[7])
    for i in range(1,len(df)):
        if ((df.iloc[i][6]-df.iloc[i-1][6])>=0):
            df.set_value(i, '7', "growth")
        else:
            df.set_value(i, '7', 'decline')
    '''Analize that we have only one recession at current data retrieval'''
    #declines=0
    #event=0
    #for i in range(1,len(df)):
    #    if df.iloc[i]['7']=="decline":
    #        declines+=1
    #        if declines==2:
    #            event+=1
    #    else:
    #        declines=0
    #print ("Number of events",event)
    declines=0
    rec_start=df.iloc[0][4]
    for i in range(1,len(df)):
        if df.iloc[i]['7']=="decline":
            declines+=1
            if declines==2:
                rec_start=df.iloc[i-1][4]
        else:
            declines=0
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    
    return rec_start

get_recession_start()

'2008q3'

In [5]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    df=pd.read_excel("gdplev.xls", skiprows=219, header=None)
    del (df[0],df[1],df[2],df[3],df[5],df[7])
    for i in range(1,len(df)):
        if ((df.iloc[i][6]-df.iloc[i-1][6])>=0):
            df.set_value(i, '7', "growth")
        else:
            df.set_value(i, '7', 'decline')
    rec_start = get_recession_start()
    start_index=df[df[4] == rec_start].index
    while ((df.iloc[start_index]["7"] == "decline").bool()):
         start_index += 1
    '''Check that next two would be growth'''
    if ((df.iloc[start_index+1]["7"] == "growth").bool()):
        end_index = start_index+1
    else:
        end_index = "It's not a recession"
    rec_end=df.iloc[end_index[0]][4]
    return rec_end

get_recession_end()

'2009q4'

In [6]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    df=pd.read_excel("gdplev.xls", skiprows=219, header=None)
    del (df[0],df[1],df[2],df[3],df[5],df[7])
    for i in range(1,len(df)):
        if ((df.iloc[i][6]-df.iloc[i-1][6])>=0):
            df.set_value(i, '7', "growth")
        else:
            df.set_value(i, '7', 'decline')
    rec_start = get_recession_start()
    start_index=df[df[4] == rec_start].index
    minimum=df.iloc[start_index[0]][6]
    bottom_index=start_index
    while ((df.iloc[start_index]["7"] == "decline").bool()):
        if minimum>=df.iloc[start_index[0]][6]:
            minimum=df.iloc[start_index[0]][6]
            bottom_index=start_index
        start_index += 1
    return df.iloc[bottom_index[0]][4]

get_recession_bottom()

'2009q2'

In [65]:
def convert_housing_data_to_quarters():
    df=pd.read_csv("City_Zhvi_AllHomes.csv")
    df=df.drop(df.columns[3:51], axis=1)
    df=df.drop(df.columns[[0]], axis=1)
    df['State']=df['State'].map((lambda x: states[x]))
    df=df.sort(['RegionName'], ascending=False)
    df=df.sort(['State'], ascending=True)
    #df=df.reset_index(['State'],['RegionName'])
    df=df.set_index(['State','RegionName'])
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    for i in range (0, df.shape[1],3):
        year=2000+i//12
        quarter=((i % 12)//3) + 1
        cur_yq=str(year)+"q"+str(quarter)
        df[cur_yq]=df[df.columns[i]]+df[df.columns[i+1]]+df[df.columns[i+2]]
        df[cur_yq]=df[cur_yq].map(lambda x: x/3)
    df=df.drop(df.columns[0:200], axis=1)
    return df

convert_housing_data_to_quarters()

/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:6: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:7: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


2000q1         2000q2         2000q3  \
State     RegionName                                                          
Alabama   Creola                 77066.666667   78233.333333   78800.000000   
          Pinson                 89466.666667   89166.666667   90933.333333   
          Center Point           80966.666667   81233.333333   81500.000000   
          Duncanville           108100.000000  112033.333333  116133.333333   
          Warrior                66500.000000   65266.666667   64433.333333   
          Deatsville            112733.333333  113500.000000  112133.333333   
          Shelby                112500.000000  112400.000000  114266.666667   
          Mobile                 78233.333333   79366.666667   80433.333333   
          Clay                  120900.000000  122266.666667  123966.666667   
          Gardendale            114200.000000  114366.666667  114766.666667   
          Ramer                           NaN            NaN            NaN   
          Crane Hill                      NaN            NaN            NaN   
          Gulf Shores           156500.000000  158900.000000  165700.000000   
          Baileyton                       NaN            NaN            NaN   
          Robertsdale            86833.333333   87666.666667   89000.000000   
          Summerdale             91266.666667   91433.333333   96633.333333   
          Mountain Brook        324866.666667  329900.000000  335600.000000   
          Semmes                 88366.666667   89966.666667   90333.333333   
          Seminole              113733.333333  123300.000000  128366.666667   
          Axis                            NaN            NaN            NaN   
          Mount Vernon           49533.333333   52433.333333   51200.000000   
          Good Hope                       NaN            NaN            NaN   
          Cottondale            100833.333333  102633.333333  104766.666667   
          Birmingham             54033.333333   54400.000000   54966.666667   
          Silverhill            101200.000000  104300.000000  105333.333333   
          Wetumpka                        NaN            NaN            NaN   
          Brent                  92933.333333   94333.333333   96166.666667   
          Decatur                         NaN            NaN            NaN   
          Woodstock             107500.000000  108200.000000  108166.666667   
          Minor                  61100.000000   60033.333333   60933.333333   
...                                       ...            ...            ...   
Wisconsin Oakfield               96933.333333   99366.666667  101766.666667   
          Town of Oconomowoc    171966.666667  171166.666667  170000.000000   
          Phelps                          NaN            NaN            NaN   
          Town of Menasha        99900.000000  104366.666667  109433.333333   
          Hudson                139000.000000  158933.333333  167233.333333   
          Town of Merton        247800.000000  244933.333333  244433.333333   
          Weston                 80166.666667   82700.000000   84166.666667   
          Fall Creek            117500.000000  119533.333333  119233.333333   
          Howards Grove         114600.000000  118066.666667  122533.333333   
          Fall River            119333.333333  119666.666667  121533.333333   
          Town of Milton        106266.666667  107166.666667  107533.333333   
          Town of Oregon        159933.333333  162433.333333  163700.000000   
          Farmington                      NaN            NaN            NaN   
          Howard                107933.333333  111666.666667  117866.666667   
          Houlton               145400.000000  167966.666667  177300.000000   
          Hortonville            77000.000000   76400.000000   75566.666667   
          Horicon                94100.000000   94766.666667   94633.333333   
          Town of Neenah         86966.666667   90633.333333   94333.333333   
          

In [117]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    bottom = get_recession_bottom()
    start = get_recession_start()
    housing_data = convert_housing_data_to_quarters()
    ut = get_list_of_university_towns()
    housing_data['Ratio'] = housing_data[start].div(housing_data[bottom])
    tuple_list=[tuple(x) for x in ut.to_records(index=False)]
    university_towns = housing_data.loc[tuple_list]
    non_university_towns = housing_data.loc[~housing_data.index.isin(tuple_list)]
    rez=stats.ttest_ind(university_towns['Ratio'], non_university_towns['Ratio'],nan_policy='omit')
    pvalue=rez[1]
    meandif=rez[0]
    if meandif<0:
        answer="university town"
    else:
        answer="non-university town"
    return (pvalue<0.01),(pvalue),(answer)

run_ttest()


/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:6: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:7: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


(False, 0.013733250828045766, 'university town')